# Анализ данных NGS. Домашнее задание № 5

Выполнил: Олег Вавулов

In [1]:
!ls ../../data/hw_5

ECOLI_IS220_QUAKE_1K_paired_reads.fasta s_6.first100000.fastq.gz
ECOLI_IS220_QUAKE_1K_single_reads.fasta test1.fasta
s_6.first1000.fastq.gz                  test2.fasta
s_6.first10000.fastq.gz


In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import os
from debruijn import *

DATA_PATH = "../../data/hw_5"
KMER_LEN = 55
# READS_PATH = os.path.join(DATA_PATH, "ECOLI_IS220_QUAKE_1K_single_reads.fasta")
READS_PATH = os.path.join(DATA_PATH, "s_6.first1000.fastq.gz")
COV_CUTOFF = 50
LEN_CUTOFF = 50

# De Bruijn graph

Compressed De Bruijn graph buiding

In [3]:
# de Bruijn graph buiding
adj_matrix_full, kmer_coverage = build_debruijn_graph(READS_PATH, KMER_LEN)

4108it [00:01, 3283.17it/s]


In [4]:
%%time
# create edges table
edges_full = get_edges(adj_matrix_full)

CPU times: user 2.84 ms, sys: 183 µs, total: 3.03 ms
Wall time: 2.97 ms


In [5]:
%%time
edges = make_graph_compression(edges_full)

100%|██████████| 1864/1864 [00:04<00:00, 375.87it/s]

CPU times: user 4.93 s, sys: 92.5 ms, total: 5.03 s
Wall time: 4.98 s


In [120]:
from numba import njit, prange, float64

In [112]:
@njit
def fast_compression(edges, uninformative_nodes):
    for i in prange(len(uninformative_nodes)):
        node = uninformative_nodes[i]
        # select edges to merge
        drop_idxs = []
        drop_idxs.append(np.where(edges[:, 1] == node)[0][0])
        drop_idxs.append(np.where(edges[:, 0] == node)[0][0])
        merge = edges[drop_idxs, :]

        in_kmer = merge[0, 0]
        out_kmer = merge[1, 1]

        # drop them at first
        edges = np.delete(edges, drop_idxs, axis=0)

        # and introduce new merged edge
        new_edge = int(str(merge[0, 2])[:-kmer_len]+\
                   str(merge[1, 2]))
        new_edge = np.array([in_kmer, out_kmer, new_edge]).reshape(1, -1)
        edges = np.concatenate([edges, new_edge], axis=0)
    return edges

In [121]:
mapp = {"A": "1", "C": "2", "T": "3", "G": "4"}
rev_mapp = {x: y for y, x in mapp.items()}
def encode(node):
    return float64(int("".join(map(lambda x: mapp[x], list(node))))/10**20)

In [122]:
%%time
edges = deepcopy(edges_full)
kmer_len = len(edges.iloc[0, 0])
source_nodes = edges.groupby("source")["edge"].count()
source_nodes = source_nodes[source_nodes == 1].keys().to_list()
target_nodes = edges.groupby("target")["edge"].count()
target_nodes = target_nodes[target_nodes == 1].keys().to_list()
uninformative_nodes = list(set(source_nodes) & set(target_nodes))
# search through the graph until no uninformative node left
edges = np.array(list(map(encode, edges.values.flatten()))).reshape(-1, 3)
uninformative_nodes = list(map(encode, uninformative_nodes))
edges = fast_compression(edges, uninformative_nodes)
edges = pd.DataFrame(edges, columns=["source", "target", "edge"])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 2d, C), Tuple(list(int64)<iv=None>, slice<a:b>))
 
There are 22 candidate implementations:
[1m   - Of which 20 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(array(float64, 2d, C), Tuple(list(int64)<iv=None>, slice<a:b>))':[0m
[1m    No match.[0m
[1m   - Of which 2 did not match due to:
   Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 162.
     With argument(s): '(array(float64, 2d, C), Tuple(list(int64)<iv=None>, slice<a:b>))':[0m
[1m    Rejected as the implementation raised a specific error:
      TypeError: unsupported array index type list(int64)<iv=None> in Tuple(list(int64)<iv=None>, slice<a:b>)[0m
  raised from /Users/a18264698/.local/lib/python3.7/site-packages/numba

In [26]:
# calculate additional stat on edges
edges = add_edges_statistics(edges, KMER_LEN, kmer_coverage)

In [27]:
edges.shape

(14, 5)

In [28]:
# results saving
res_path = "results/full"
save_dot(edges, res_path, "graph.dot", "nodes.json")
dot2png(res_path, "graph.dot", "graph.png")
edges2fasta(edges, res_path, "edges.fa")

<img src="results/full/graph.png">

Tips removal

In [8]:
# tips removal
adj_matrix_notips, edges_notips = remove_tips(
    adj_matrix, edges, len_cutoff=LEN_CUTOFF, cov_cutoff=COV_CUTOFF
)

In [9]:
# results saving
res_path = "results/notips"
save_dot(edges_notips, res_path, "graph_notips.dot", "nodes_notips.json")
dot2png(res_path, "graph_notips.dot", "graph_notips.png")
edges2fasta(edges_notips, res_path, "edges_notips.fa")

<img src="results/notips/graph_notips.png">

Any unreliable edges removal

In [16]:
# tips removal
adj_matrix_noany, edges_noany = remove_any(
    adj_matrix, edges, len_cutoff=LEN_CUTOFF, cov_cutoff=COV_CUTOFF
)

In [22]:
# results saving
res_path = "results/noany"
save_dot(edges_noany, res_path, "graph_noany.dot", "nodes_noany.json")
dot2png(res_path, "graph_noany.dot", "graph_noany.png")
edges2fasta(edges_noany, res_path, "edges_noany.fa")

<img src="results/noany/graph_noany.png">

При обеих стратегиях очистки графа получаем один и тот же результат, однако способ с удалением хвостиков более аккуратный, так как при его использовании гарантированно сохраняется число компонент связности в графе.